# Working With STAC

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/developmentseed/titiler/master?filepath=docs%2Fexamples%2FWorking_with_STAC_simple.ipynb)

### STAC: SpatioTemporal Asset Catalog

> The SpatioTemporal Asset Catalog (STAC) specification aims to standardize the way geospatial assets are exposed online and queried. A 'spatiotemporal asset' is any file that represents information about the earth captured in a certain space and time. The initial focus is primarily remotely-sensed imagery (from satellites, but also planes, drones, balloons, etc), but the core is designed to be extensible to SAR, full motion video, point clouds, hyperspectral, LiDAR and derived data like NDVI, Digital Elevation Models, mosaics, etc.

Ref: https://github.com/radiantearth/stac-spechttps://github.com/radiantearth/stac-spec

Using STAC makes data indexation and discovery really easy. In addition to the Collection/Item/Asset (data) specifications, data providers are also encouraged to follow a STAC API specification:  https://github.com/radiantearth/stac-api-spec

> The API is compliant with the OGC API - Features standard (formerly known as OGC Web Feature Service 3), in that it defines many of the endpoints that STAC uses. A STAC API should be compatible and usable with any OGC API - Features clients. The STAC API can be thought of as a specialized Features API to search STAC Catalogs, where the features returned are STAC Items, that have common properties, links to their assets and geometries that represent the footprints of the geospatial assets.


## Requirements

To be able to run this notebook you'll need the following requirements:
- folium
- requests
- rasterio

`!pip install folium requests rasterio`

In [ ]:
# Uncomment this line if you need to install the dependencies
# !pip install folium requests rasterio

In [ ]:
import requests

from rasterio.features import bounds as featureBounds

from folium import Map, TileLayer, GeoJson

%pylab inline

In [ ]:
titiler_endpoint = "https://api.cogeo.xyz"  # Devseed temporary endpoint
stac_item = "https://canada-spot-ortho.s3.amazonaws.com/canada_spot_orthoimages/canada_spot5_orthoimages/S5_2007/S5_11055_6057_20070622/S5_11055_6057_20070622.json"

In [ ]:
item = requests.get(stac_item).json()
print(item)

In [ ]:
print(list(item["assets"]))
for it, asset in item["assets"].items():
    print(asset["type"])

In [ ]:
bounds = featureBounds(item)

m = Map(
    tiles="OpenStreetMap",
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=8
)

geo_json = GeoJson(data=item)
geo_json.add_to(m)
m

In [ ]:
# Get Tile URL
r = requests.get(
    f"{titiler_endpoint}/stac/info",
    params = {
        "url": stac_item,
    }
).json()
print(r)

### Display one asset

In [ ]:
r = requests.get(
    f"{titiler_endpoint}/stac/tilejson.json",
    params = {
        "url": stac_item,
        "assets": "pan",
        "minzoom": 8,  # By default titiler will use 0
        "maxzoom": 14, # By default titiler will use 24
    }
).json()

m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=10
)

tiles = TileLayer(
    tiles=r["tiles"][0],
    min_zoom=r["minzoom"],
    max_zoom=r["maxzoom"],
    opacity=1,
    attr="Government of Canada"
)
tiles.add_to(m)
m

### Merge Assets

In [ ]:
# Get Tile URL
r = requests.get(
    f"{titiler_endpoint}/stac/tilejson.json",
    params = {
        "url": stac_item,
        "assets": "B3,B2,B1",  # Simple RGB combination
        "minzoom": 8,  # By default titiler will use 0
        "maxzoom": 14, # By default titiler will use 24
    }
).json()

m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=10
)

tiles = TileLayer(
    tiles=r["tiles"][0],
    min_zoom=r["minzoom"],
    max_zoom=r["maxzoom"],
    opacity=1,
    attr="Government of Canada"
)
tiles.add_to(m)
m